In [ ]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

sys.path.append(os.path.abspath('..'))
from src.utils.seed import set_seed
from src.utils.config import Config
from src.preprocessing import get_data_loaders
from src.models import LogisticRegression
from src.evaluation import calculate_metrics, Plotter

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

c:\Users\surya\anaconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda


In [2]:
config = Config('../configs/config.yaml')
train_loader, val_loader, test_loader, input_dim = get_data_loaders(config)

print(f"Input Features: {input_dim}")
print(f"Train Samples: {len(train_loader.dataset)}")
print(f"Val Samples: {len(val_loader.dataset)}")
print(f"Test Samples: {len(test_loader.dataset)}")

Input Features: 7
Train Samples: 130812
Val Samples: 18688
Test Samples: 37376


In [3]:
model = LogisticRegression(input_dim=input_dim).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=config.training['learning_rate'])

print(f"Model: {model}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

Model: LogisticRegression(
  (linear): Linear(in_features=7, out_features=1, bias=True)
)
Parameters: 8


In [4]:
epochs = config.training['epochs']
train_losses = []
val_losses = []
val_aucs = []

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    for X_batch, y_batch in progress_bar:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device).float().unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_train_loss = train_loss / len(train_loader)
    
    model.eval()
    val_loss = 0.0
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device).float().unsqueeze(1)
            
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()
            
            probs = torch.sigmoid(outputs)
            all_labels.extend(y_batch.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    avg_val_loss = val_loss / len(val_loader)
    val_metrics = calculate_metrics(np.array(all_labels), np.array(all_probs))
    
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    val_aucs.append(val_metrics['auc'])
    
    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val AUC: {val_metrics['auc']:.4f}")

print("\n✓ Training Complete!")

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(range(1, epochs+1), train_losses, label='Train Loss', marker='o')
axes[0].plot(range(1, epochs+1), val_losses, label='Val Loss', marker='s')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Baseline Model: Training Progress')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(range(1, epochs+1), val_aucs, label='Val AUC', marker='o', color='green')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('AUC')
axes[1].set_title('Validation AUC')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
model.eval()
all_preds = []
all_probs = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        probs = torch.sigmoid(outputs).cpu().numpy()
        
        all_probs.extend(probs.flatten())
        all_labels.extend(y_batch.numpy())

all_probs = np.array(all_probs)
all_labels = np.array(all_labels)

test_metrics = calculate_metrics(all_labels, all_probs)

print("Baseline Model Test Results:")
for metric, value in test_metrics.items():
    print(f"  {metric.upper()}: {value:.4f}")

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

fpr, tpr, _ = roc_curve(all_labels, all_probs)
roc_auc = auc(fpr, tpr)

precision, recall, _ = precision_recall_curve(all_labels, all_probs)
pr_auc = auc(recall, precision)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(fpr, tpr, lw=2, label=f'ROC (AUC = {roc_auc:.3f})')
axes[0].plot([0, 1], [0, 1], 'k--', lw=2, label='Random')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve - Baseline Model')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(recall, precision, lw=2, label=f'PR (AUC = {pr_auc:.3f})')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
os.makedirs('../reports/results/checkpoints', exist_ok=True)
torch.save(model.state_dict(), '../reports/results/checkpoints/baseline_model.pth')
print("✓ Model saved to: reports/results/checkpoints/baseline_model.pth")